In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "5,6"  # 指定使用 GPU 3
import sys
sys.path.append('/home/mcq/GitHub/aes2')
sys.path.append('/home/mcq/GitHub/aes2/topic-classificatio')
import aes2_added_fb_prize_as_features_preprocessing
import gc

gc.collect()


CUDA_AVAILABLE = True


1682

In [5]:
import numpy as np
from sklearn.model_selection import GroupKFold
import pandas as pd
import topicfeaturesmall
import pickle

with open("/home/mcq/GitHub/aes2/train_data/train_feats.pkl", "rb") as f:
    train_feats = pickle.load(f)
with open("/home/mcq/GitHub/aes2/train_data/X.pkl", "rb") as f:
    X = pickle.load(f)
with open("/home/mcq/GitHub/aes2/train_data/y.pkl", "rb") as f:
    y = pickle.load(f)
with open("/home/mcq/GitHub/aes2/train_data/y_split.pkl", "rb") as f:
    y_split = pickle.load(f)
with open(
    "/home/mcq/GitHub/aes2/train_data/feature_select.pkl", "rb"
) as f:
    feature_select = pickle.load(f)
    
aes2_added_fb_prize_as_features_preprocessing.feature_select = feature_select

PATH = "/home/mcq/GitHub/aes2/kaggle/input/learning-agency-lab-automated-essay-scoring-2/"
train = pd.read_csv(PATH + "train.csv")

topic = topicfeaturesmall.predict_chunk(train)

groups = topic['topic'] ### 这个groups是分类依据，原理是返回这个group对应的index


Map:   0%|          | 0/17307 [00:00<?, ? examples/s]

/home/mcq/anaconda3/envs/torch/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


### 加载XY

In [6]:
import numpy as np

def clean_feature_names(features):
    illegal_chars = ['[', ']', '<', '>']
    cleaned_features = []
    for feature in features:
        for char in illegal_chars:
            feature = feature.replace(char, 'lessthan')
        cleaned_features.append(feature)
    return cleaned_features

train_feats.columns = clean_feature_names(train_feats.columns)
feature_select = clean_feature_names(feature_select)
X = train_feats[feature_select].astype(np.float32).values


# X = train_feats[feature_names].astype(np.float32).values
#X = train
y_split = train['score'].astype(int).values

In [9]:
LOAD = True 

n_splits = 7 ## 如果变更为大数据集，这个_split要更改
models = []


if not LOAD:
    for i in range(n_splits):
        models.append(lgb.Booster(model_file=f'kaggle/input/aes-lgbm/fold_{i+1}.txt'))
else:
    group_kfold = GroupKFold(n_splits=n_splits)
    f1_scores = []
    kappa_scores = []
    models = []
    predictions = []
 
    for i, (train_index, test_index) in enumerate(group_kfold.split(X, y_split, groups)):
        X_train_fold, X_test_fold = X[train_index], X[test_index]
        y_train_fold, y_test_fold, y_test_fold_int = y_split[train_index], y_split[test_index], y_split[test_index]
        # 可以通过这三行代码看一下数据，直接删掉就可以
        print(f"Fold {i}:\n\n")
        print(f"  Train: index={train_index}\n group={groups[train_index]}\n\n")
        print(f"  Test:  index={test_index}\n group={groups[test_index]}\n\n")



Fold 0:


  Train: index=[    0     1     3 ... 17304 17305 17306]
 group=0        1
1        6
3        4
4        2
5        2
        ..
17302    4
17303    5
17304    0
17305    4
17306    4
Name: topic, Length: 13822, dtype: int64


  Test:  index=[    2    10    12 ... 17282 17286 17291]
 group=2        3
10       3
12       3
14       3
19       3
        ..
17271    3
17277    3
17282    3
17286    3
17291    3
Name: topic, Length: 3485, dtype: int64


Fold 1:


  Train: index=[    0     1     2 ... 17304 17305 17306]
 group=0        1
1        6
2        3
3        4
4        2
        ..
17300    0
17302    4
17304    0
17305    4
17306    4
Name: topic, Length: 14261, dtype: int64


  Test:  index=[    6    16    21 ... 17292 17301 17303]
 group=6        5
16       5
21       5
24       5
28       5
        ..
17284    5
17290    5
17292    5
17301    5
17303    5
Name: topic, Length: 3046, dtype: int64


Fold 2:


  Train: index=[    0     1     2 ... 17301 17303 17304]
 gr